# Защита персональных данных клиентов

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Датасет состоит из 5000 строк и 5 столбцов. Все значения - числовые, возраст и зарплата дробные, остальные целые, пропусков нет. Разделим данные на признаки и целевой.

In [4]:
features = df.drop('Страховые выплаты', axis = 1)
target = df['Страховые выплаты']

## Умножение матриц

**Вопрос:** признаки умножают на обратимую матрицу, изменится ли качество линейной регрессии? (Её можно обучить заново.)

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Качество линейной регрессии определяется предсказаниями *a*. Значит вопрос в том, как изменится *a*  при умножении матрицы *X* на обратимую матрицу *P*. Чтобы ответить на этот вопрос, запишем новое $a_P$ и посмотрим, как оно соотносится с исходным *a*.

Для выполнения преобразований понадобятся свойства обратных и транпcонированных матриц:

$$
(A B)^{-1} = B^{-1} A^{-1}
$$

$$
(A B)^{T} = B^{T} A^{T}
$$

Сначала рассмотрим исходный вектор предсказаний:
$$
a = Xw = X (X^T X)^{-1} X^T y 
$$

Проделаем преобразования с новым вектором предсказаний:
$$
a_P = (X P) w_P =\\
= (X P) ((X P)^T (X P))^{-1} (X P)^{T} y = \\
= X P (P^{T} X^{T} X P)^{-1} P^{T} X^{T} y =\\
= X P (P^{T} (X^{T} X P))^{-1} P^{T} X^{T} y =\\
= X P (X^{T} X P)^{-1} (P^{T})^{-1} P^{T} X^{T} y =\\
= X P ((X^{T} X) P)^{-1} (P^{T})^{-1} P^{T} X^{T} y =\\
= X P P^{-1} (X^{T} X)^{-1} (P^{T})^{-1} P^{T} X^{T} y\\
$$

$$P P^{-1} = E$$
$$(P^{T})^{-1} P^{T} = E$$

Поэтому выражение принимает вид:

$$
a_P = X E (X^{T} X)^{-1} E X^{T} y \\
$$

Умножение на единичную матрицу здесь ничего не меняет, поэтому:

$$
a_P = X (X^{T} X)^{-1} X^{T} y = a
$$

**Ответ: качество линейной регрессии не меняется при умножении признаков на обратимую матрицу.**

## Алгоритм преобразования

Для решения задачи можно предложить умножить таблицу признаков, то есть данные о клиентах, на случайную обратимую матрицу: на качество линейной регрессии это не повлияет, как мы только что видели, а для постороннего наблюдателя данные будут выглядеть бессмысленно.

Обратимая матрица - квадратная матрица. Признаки точно не квадратные, но для возможности умножения остаточно, чтобы матрица *P* имела размерность, равную количеству признаков. Из-за особенностей матричного умножения структура датасета не поменяется, т.е. первая строка так и будет содержать данные о первом клиенте.

In [5]:
def transform_alg(features):  
   
    #создаем случайную матрицу
    P = np.random.randint (-10, 10, (features.shape[1], features.shape[1]))
    
    #проверяем матрицу на обратимость
    try:
        np.linalg.inv(P)
    except:
        print('сгенерированная матрица необратима')
        return None
        
    #трансформируем признаки
    features_transformed = features @ P
     
    return features_transformed

## Проверка алгоритма

Чтобы убедиться, что все работает, сделаем предсказания линейной регрессией сначала на исходных данных, а потом на преобразованных, и сравним результаты.

In [6]:
features_transformed = transform_alg(features)
features_transformed

,0,1,2,3
0,346831.0,-99365.0,-347278.0,49312.0
1,265585.0,-76182.0,-266085.0,37679.0
2,146739.0,-42116.0,-147058.0,20797.0
3,291709.0,-83480.0,-291928.0,41555.0
4,182449.0,-52315.0,-182759.0,25902.0
...,...,...,...,...
4995,249646.0,-71508.0,-249942.0,35506.0
4996,366493.0,-104934.0,-366861.0,52163.0
4997,237118.0,-67876.0,-237326.0,33762.0
4998,228700.0,-65485.0,-228926.0,32547.0


Преобразованные данные выглядят бессмысленным набором чисел без подписей. То, что надо.

In [7]:
def linear_regression(features, target):
    
    model=LinearRegression()
    model.fit(features, target)
    return model.predict(features)

In [8]:
print('R2 до преобразования:\n', r2_score(target, linear_regression(features, target)))
print('R2 после преобразования:\n', r2_score(target, linear_regression(features_transformed, target)))

R2 до преобразования:
 0.4249455028666801
R2 после преобразования:
 0.4249455028666783


Качество линейной регрессии действительно не изменилось: метрики равны до 11-13 знака после запятой. 

*(random_state не указан специально, чтобы показать, что r2 равны вне зависимости от конкретных значений чисел в матрице.)*

**Вывод: метод защиты персональных данных умножением их на обратимую матрицу работает: данные выглядят непонятно для человека, а линейная регрессия работает так же.**